In [1]:
import sys
import numpy as np
import sklearn_crfsuite
import time
from tqdm import tqdm
import pandas as pd
import prepare_train_with_set as prepare
import os
import func

In [ ]:
if __name__ == "__main__":
    
    set_total = 0
    # How many Set
    ALL = True
    if not ALL:
        set_num = 1
    DEBUG = False
    
    current_path = os.path.join(os.path.expanduser("~"), "jupyter", "web_verification")
    data, Set_idx = prepare.train_file_generate(set_total, current_path)
    
    max_num_train = func.load_data_num(data)
    
    x_train, y_train, _ = func.CRFSuite_proccess_data(data)

# Check max_num in train

Open the train file and test file to check max number of nodes for each page to give the number for padding.

In [ ]:
max_num_train, max_label_train = load_data_num("./data/train_raw.csv", True)
max_num_test = load_data_num("./data/ytest_raw.csv", False)
max_num = max(max_num_train, max_num_test)
if DEBUG:
    print(max_num_train)
    print(max_num_test)

# Load Set index File

Open Set_idx file to check which label is a Set in the training file generated by the training file generation part.

In [ ]:
col_set_dict={}
if set_total > 0:
    Set_dict = {}
    with open("./data/Set_idx.txt", "r") as set_file:
        Set_dict = eval(set_file.readline())
    col_set_dict = dict(map(reversed, Set_dict.items()))
    if DEBUG:
        print(Set_dict)

# Load Train File

Read Training file and convert into CRF features.

In [ ]:
max_label = max_label_train
df = get_df("./data/train_raw.csv")
feature_train, label_train, out_train = load_data_csv(df)
page_num = int(len(feature_train)/max_num)
#input_shape = feature_train.shape[1]

In [ ]:
crf = sklearn_crfsuite.CRF(
    c1=0.1,
    max_iterations=50
)
feature_train = feature_train.tolist()
label_train = label_train.tolist()
X_train = [sent2features(feature_train)]
y_train = [sent2labels(label_train)]

# Train

Start training and recording the time consumption.

In [ ]:
start = time.time()
crf.fit(X_train, y_train)
t = time.time()-start

# Load Test file

In [ ]:
df = get_df("./data/ytest_raw.csv")
feature_test, label_test, out_test = load_data_csv(df)

feature_test = feature_test.tolist()
label_test = label_test.tolist()
X_test = [sent2features(feature_test)]
page_test = int(len(feature_test)/max_num)
if DEBUG:
    print(page_test)

# Prediction

Predict on test file and record the testing time.

In [ ]:
labels = list(crf.classes_)
ts_start = time.time()
y_pred = crf.predict(X_test)
ts = time.time()-ts_start
y_pred = np.array(y_pred)
result = np.reshape(y_pred, [page_test, max_num])
result = result.astype(np.int64)

# Read Column Type

Read Column type from TableA for file ColType output.

In [ ]:
col_type = []
with open("./data/TableA.txt", "r") as file:
    line = file.readline()
    slot = line.rstrip("\n").split("\t")
    while(slot[0]!="ColType"):
        line = file.readline()
        slot = line.rstrip("\n").split("\t")
    col_type = slot[1:]
if DEBUG:
    print(col_type)

# File prediction output

Output the prediction.csv file for use.

In [ ]:
Set_data = []
with open("./crfsuite/data/predictions.csv", "w") as file: # Create prediction file
    for col in col_type: # loop to write the Col type
        file.write(col + "\t")
        if DEBUG:
            print(col + "\t", end='')
    file.write("\n")
    for page in tqdm(range(len(result))): # Loop each page
        sets = []
        for label in range(max_label + 1): # Loop whole label
            if DEBUG:
                print("Label: " + str(label))
            if label == 0:
                continue
            empty = True
            isset = False
            data = []
            for node in range(len(result[page])):
                if result[page][node] == label:
                    if empty == False and not isset:
                        if DEBUG:
                            print(" ", end='')
                        file.write(" ")
                    empty = False
                    if label in col_set_dict.keys() and set_total > 0: # That col is a Set
                        isset = True
                        data.append(node)
                        if DEBUG:
                            print("Append:" + str(node))
                    else:
                        if DEBUG:
                            print(str(node), end='')
                        file.write(str(node))
            if label in col_set_dict.keys() and set_total > 0: # That col is a Set
                if DEBUG:
                    print(str(col_set_dict[label])+"-"+str(page), end='')
                file.write(str(col_set_dict[label])+"-"+str(page))
                sets.append(data)
            if DEBUG:
                print("\t", end='')
            file.write("\t")
        if DEBUG:
            print("")
        file.write("\n")
        if DEBUG:
            print(data)
        Set_data.append(sets)
print("Done!")

# Create Set data output for test

Output the Set data that being predicted in the Set by the model.

In [ ]:
if set_total > 0:
    with open("./crfsuite/set/Set_data.txt", "w") as set_train_file:
        tmp = str(Set_data)
        set_train_file.write(tmp)
        if DEBUG:
            print(tmp)

# Create Set Train File

Generate train file for Set Model from DCADE Set Table.

In [ ]:
set_data_count = []
if set_total > 0:
    for set_t in range(set_total):
        with open("./data/Set-"+ str(set_t+1) +".txt", "r") as set_file:
            set_tmp = []
            output_name = "./crfsuite/set/Set-"+ str(set_t+1) +"_train_raw.csv"
            if DEBUG:
                print("Generating:" + output_name + "\n")
            output = open(output_name, "w")
            output.write("Leafnode\tPTypeSet\tTypeSet\tContentid\tPathid\tSimseqid\tLabel\n")
            line = set_file.readline()
            slot = line.rstrip("\n").split("\t")
            while(slot[0]!="ColType"): 
                line = set_file.readline()
                slot = line.rstrip("\n").split("\t")
            with open("./set/Set-"+ str(set_t+1) +"_coltype.txt", "w") as col_file:
                col_file.write(str(slot[1:]))
            line = set_file.readline() # First line of data
            page_num = 0
            count = 0
            while(line != ""):
                slot = line.rstrip("\n").split("\t")
                data_info = slot[0].split("-")
                if(page_num != int(data_info[1])):
                    set_tmp.append(count)
                    count = 0
                set_num = int(data_info[0])
                page_num = int(data_info[1])
                if DEBUG:
                    print(str(data_info[0])+"-"+str(data_info[1])+"-"+str(data_info[2]))
                idx = 1
                sub_list = slot[1:]
                while("" in sub_list):
                    sub_list.remove("")
                while(" " in sub_list):
                    sub_list.remove(" ")
                for element in sub_list:
                    count += 1
                    if DEBUG:
                        print(element)
                    element = int(element)
                    #print(content_train[page_num][element])
                    output.write(str(feature_train[page_num*max_num+element][0])+"\t")
                    output.write(str(feature_train[page_num*max_num+element][1])+"\t")
                    output.write(str(feature_train[page_num*max_num+element][2])+"\t")
                    output.write(str(feature_train[page_num*max_num+element][3])+"\t")
                    output.write(str(feature_train[page_num*max_num+element][4])+"\t")
                    output.write(str(feature_train[page_num*max_num+element][5])+"\t")
                    output.write(str(idx) + "\n")
                    if DEBUG:
                        print(feature_train[page_num*max_num+element][0])
                    idx += 1
                line = set_file.readline()
            set_tmp.append(count)
            output.close()
        set_data_count.append(set_tmp)

In [ ]:
if set_total > 0:
    with open("./crfsuite/set/set_train_count.txt", "w") as file:
        file.write(str(set_data_count))
        if DEBUG:
            print(set_data_count)

# Create Set Test file

Generate test file from node data being predicted in a Set by model.

In [ ]:
set_data_count = []
if set_total > 0:
    for set_t in range(set_total):
        set_tmp = []
        with open("./crfsuite/set/Set-"+ str(set_t+1) +"_ytest_raw.csv", "w") as set_file:
            set_file.write("Leafnode\tPTypeSet\tTypeSet\tContentid\tPathid\tSimseqid\tLabel\n")
            for pages in tqdm(range(len(Set_data))):
                count = 0
                for node in Set_data[pages][set_t]:
                    count += 1
                    set_file.write(str(feature_test[pages*max_num+node][0])+"\t")
                    set_file.write(str(feature_test[pages*max_num+node][1])+"\t")
                    set_file.write(str(feature_test[pages*max_num+node][2])+"\t")
                    set_file.write(str(feature_test[pages*max_num+node][3])+"\t")
                    set_file.write(str(feature_test[pages*max_num+node][4])+"\t")
                    set_file.write(str(feature_test[pages*max_num+node][5])+"\t")
                    set_file.write(str(0) + "\n")
                set_tmp.append(count)
        set_data_count.append(set_tmp)

In [ ]:
if set_total > 0:
    with open("./crfsuite/set/set_test_count.txt", "w") as file:
        file.write(str(set_data_count))
        if DEBUG:
            print(set_data_count)

In [ ]:
page_c = len(result)

# Set function define

In [ ]:
def max_num_set(set_data_count, set_total):
    max_set = []
    for i in range(set_total):
        max_set.append(0)
    for sets in range(len(set_data_count)):
        max_set[sets] = max(set_data_count[sets])
    return max_set

def feature_padding_set(df, set_count, set_num):
    feature = []
    count = 0
    for pages in set_count[set_num-1]:
        set_len = pages
        for i in range(set_len):
            feature.append(df[count])
            count += 1
        if set_len != max_set[set_num-1]:
            for i in range(max_set[set_num-1]-set_len):
                feature.append(9999)
    return feature

def one_of_n(ans, total):
    tmp = []
    for i in range(int(total)):
        if ans == i:
            tmp.append(1.0)
        else:
            tmp.append(0.0)
    return tmp

def label_padding_set(df, set_count, set_num):
    label = []
    count = 0
    for pages in set_count[set_num-1]:
        set_len = pages
        for i in range(set_len):
            label.append(df[count])
            count += 1
        if set_len != max_set[set_num-1]:
            for i in range(max_set[set_num-1]-set_len):
                label.append(0)
    return label

def to_train_array_set(df, set_count, set_num):
    feature_1 = np.array(feature_padding_set(df['Leafnode'], set_count, set_num))
    feature_2 = np.array(feature_padding_set(df['PTypeSet'], set_count, set_num))
    feature_3 = np.array(feature_padding_set(df['TypeSet'], set_count, set_num))
    feature_4 = np.array(feature_padding_set(df['Contentid'], set_count, set_num))
    feature_5 = np.array(feature_padding_set(df['Pathid'], set_count, set_num))
    feature_6 = np.array(feature_padding_set(df['Simseqid'], set_count, set_num))
    
    feature_1 = np.expand_dims(feature_1, -1)
    feature_2 = np.expand_dims(feature_2, -1)
    feature_3 = np.expand_dims(feature_3, -1)
    feature_4 = np.expand_dims(feature_4, -1)
    feature_5 = np.expand_dims(feature_5, -1)
    feature_6 = np.expand_dims(feature_6, -1)
    
    feature = np.concatenate([feature_1, feature_2, feature_3, feature_4, feature_5, feature_6], -1)
    feature = np.reshape(feature, [len(set_count[set_num-1])*max_set[set_num-1], 6])
    
    label = np.array(label_padding_set(df['Label'], set_count, set_num))
    label = np.reshape(label, [len(set_count[set_num-1])*max_set[set_num-1]])
    return feature, label

# Read Set data

In [ ]:
if set_total > 0:
    Set_data = []
    set_train_count = []
    set_test_count = []
    with open("./crfsuite/set/Set_data.txt", "r") as set_file:
        Set_data = eval(set_file.readline())
    with open("./crfsuite/set/set_train_count.txt", "r") as set_file:
        set_train_count = eval(set_file.readline())
    with open("./crfsuite/set/set_test_count.txt", "r") as set_file:
        set_test_count = eval(set_file.readline())
    max_num_train = max_num_set(set_train_count, set_total)
    max_num_test = max_num_set(set_test_count, set_total)
    max_set = []
    for i in range(len(max_num_train)):
        max_set.append(max(max_num_train[i], max_num_test[i]))
    if DEBUG:
        print(max_num_train)
        print(max_num_test)
        print(max_set)

# Run ALL

In [ ]:
if set_total > 0:
    for num in range(set_total):
        set_num = num + 1
        # Load Train file & Test file
        df = get_df("./crfsuite/set/Set-"+str(set_num)+"_train_raw.csv")
        max_num = max_set[set_num-1]
        max_label = max(df['Label'])
        feature_train, label_train = to_train_array_set(df, set_train_count, set_num)
        page_num = int(len(feature_train)/max_num)
        crf = sklearn_crfsuite.CRF(
            c1=0.1
        )
        feature_train = feature_train.tolist()
        label_train = label_train.tolist()
        X_train = [sent2features(feature_train)]
        y_train = [sent2labels(label_train)]
        
        # Train
        start = time.time()
        crf.fit(X_train, y_train)
        t += time.time()-start
        
        # Load Test file
        df = get_df("./crfsuite/set/Set-"+str(set_num)+"_ytest_raw.csv")
        feature_test, label_test = to_train_array_set(df, set_test_count, set_num)
        feature_test = feature_test.tolist()
        label_test = label_test.tolist()
        X_test = [sent2features(feature_test)]
        page_test = int(len(feature_test)/max_num)
            
        # Prediction
        labels = list(crf.classes_)
        ts_start = time.time()
        y_pred = crf.predict(X_test)
        ts += time.time()-ts_start
        y_pred = np.array(y_pred)
        result = np.reshape(y_pred, [page_test, max_num])
        result = result.astype(np.int64)
        
        # Read Col
        col_type = []
        with open("./set/Set-"+str(set_num)+"_coltype.txt", "r") as file:
            tmp = file.readline()
            slot = eval(tmp)
            col_type = slot
            
        # Output
        Set = []
        with open("./crfsuite/set/set-"+str(set_num)+".csv", "w") as file: # Create prediction file
            for col in col_type: # loop to write the Col type
                file.write(col + "\t")
                if DEBUG:
                    print(col + "\t", end='')
            if DEBUG:
                print("")
            file.write("\n")
            current_pos = 1
            for page in tqdm(range(len(result))): # Loop each page
                p_tmp = []
                for cols in range(max_label+1):
                    c_tmp = []
                    for node in range(len(result[page])):
                        r = result[page][node]
                        if r == cols:
                            c_tmp.append(node)
                    p_tmp.append(c_tmp)
                Set.append(p_tmp)
            Set_tmp = Set.copy()
            for page in range(len(Set_tmp)):
                empty = False
                col = []
                for i in range(len(Set_tmp[page])):
                    col.append(False)
                col[0] = True
                while(not empty):
                    for cols in range(len(Set_tmp[page])):
                        if len(Set_tmp[page][cols]) == 0:
                            col[cols] = True
                            if cols != 0:
                                if DEBUG:
                                    print("\t", end="")
                                file.write("\t")
                        else:
                            n = str(int(feature_test[page*max_num+Set_tmp[page][cols][0]][0]))
                            if cols != 0:
                                if DEBUG:
                                    print(n+"\t", end="")
                                file.write(n+"\t")
                            del Set_tmp[page][cols][0]
                            if len(Set_tmp[page][cols]) == 0:
                                col[cols] = True
                        empty = True
                        for i in col:
                            if i == False:
                                empty = False
                                break
                    if DEBUG:
                        print("\n", end="")
                    file.write("\n")

In [ ]:
timef = open("./crfsuite/data/time_crf.txt","w")
print("\ntrain time:"+str(t))
timef.write("train:"+str(t)+"\n")
print("test time:"+str(ts))
print("per page:"+ str(float(ts)/page_c)+"\n")
timef.write("test:"+str(ts)+"\n")
timef.write("per page:"+ str(float(ts)/page_c)+"\n")
timef.close()

In [ ]:
#S56 500